In [38]:
import sqlalchemy
import requests
import pandas as pd
from sqlalchemy import create_engine

In [37]:
engine = create_engine("mysql+mysqlconnector://root@localhost/test", echo=True)

In [24]:
url = "https://date.nager.at/api/v2/publicholidays/2020/US"
respon = requests.get(url=url)

In [25]:
respon

<Response [200]>

In [26]:
data = respon.json()
data

[{'date': '2020-01-01',
  'localName': "New Year's Day",
  'name': "New Year's Day",
  'countryCode': 'US',
  'fixed': False,
  'global': True,
  'counties': None,
  'launchYear': None,
  'type': 'Public'},
 {'date': '2020-01-20',
  'localName': 'Martin Luther King, Jr. Day',
  'name': 'Martin Luther King, Jr. Day',
  'countryCode': 'US',
  'fixed': False,
  'global': True,
  'counties': None,
  'launchYear': None,
  'type': 'Public'},
 {'date': '2020-02-17',
  'localName': 'Presidents Day',
  'name': "Washington's Birthday",
  'countryCode': 'US',
  'fixed': False,
  'global': True,
  'counties': None,
  'launchYear': None,
  'type': 'Public'},
 {'date': '2020-04-10',
  'localName': 'Good Friday',
  'name': 'Good Friday',
  'countryCode': 'US',
  'fixed': False,
  'global': False,
  'counties': ['US-CT',
   'US-DE',
   'US-HI',
   'US-IN',
   'US-KY',
   'US-LA',
   'US-NC',
   'US-ND',
   'US-NJ',
   'US-TN'],
  'launchYear': None,
  'type': 'Public'},
 {'date': '2020-04-10',
  'loca

In [42]:
df_data = pd.DataFrame(data)
df_data['counties'] = df_data['counties'].astype(str)
df_data

,date,localName,name,countryCode,fixed,global,counties,launchYear,type
0,2020-01-01,New Year's Day,New Year's Day,US,False,True,None,NaN,Public
1,2020-01-20,"Martin Luther King, Jr. Day","Martin Luther King, Jr. Day",US,False,True,None,NaN,Public
2,2020-02-17,Presidents Day,Washington's Birthday,US,False,True,None,NaN,Public
3,2020-04-10,Good Friday,Good Friday,US,False,False,"['US-CT', 'US-DE', 'US-HI', 'US-IN', 'US-KY', ...",NaN,Public
4,2020-04-10,Good Friday,Good Friday,US,False,False,['US-TX'],NaN,Optional
5,2020-05-25,Memorial Day,Memorial Day,US,False,True,None,NaN,Public
6,2020-07-03,Independence Day,Independence Day,US,False,True,None,NaN,Public
7,2020-09-07,Labor Day,Labour Day,US,False,True,None,NaN,Public
8,2020-10-12,Columbus Day,Columbus Day,US,False,False,"['US-AL', 'US-AZ', 'US-CO', 'US-CT', 'US-GA', ...",NaN,Public
9,2020-11-11,Veterans Day,Veterans Day,US,False,True,None,NaN,Public


In [28]:
df_data.to_csv("public_holiday.csv")

In [29]:
df_data.to_json("public_holiday.json")

In [43]:
#save to table mysql
conn = engine.connect()
trans = conn.begin()

try:
    df_data.to_sql("assignment_web_scraping", 
                conn, if_exists='replace', 
                index=False)
    
    trans.commit()
except Exception as e:
    trans.rollback()
    print(f"Error: {e}")

2024-02-09 09:47:46,868 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-09 09:47:46,877 INFO sqlalchemy.engine.Engine DESCRIBE `test`.`assignment_web_scraping`
2024-02-09 09:47:46,878 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-02-09 09:47:46,911 INFO sqlalchemy.engine.Engine DESCRIBE `test`.`assignment_web_scraping`
2024-02-09 09:47:46,912 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-02-09 09:47:46,955 INFO sqlalchemy.engine.Engine SHOW FULL TABLES FROM `test`
2024-02-09 09:47:46,956 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-02-09 09:47:46,983 INFO sqlalchemy.engine.Engine SHOW FULL TABLES FROM `test`
2024-02-09 09:47:46,983 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-02-09 09:47:47,001 INFO sqlalchemy.engine.Engine SHOW CREATE TABLE `assignment_web_scraping`
2024-02-09 09:47:47,002 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-02-09 09:47:47,013 INFO sqlalchemy.engine.Engine 
DROP TABLE assignment_web_scraping
2024-02-09 09:47:47,015 INFO sqlalchemy.engine.E